In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


In [ ]:
import os
import numpy as np
import librosa

# Constants
SAMPLE_RATE = 22050  # Adjust the sample rate as needed
DURATION = 4  # Adjust the duration as needed
MFCC_FEATURES = 13  # Adjust the number of MFCC features as needed

# Paths to the audio data directories
pneumonia_audio_dir = '/content/drive/MyDrive/New Audio Folder/Pneumonia'
normal_audio_dir = '/content/drive/MyDrive/New Audio Folder/Normal'

# Function to extract MFCC features
def extract_features(file_path):
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    mfccs = librosa.feature.mfcc(y=audio, sr=SAMPLE_RATE, n_mfcc=MFCC_FEATURES)

    # Pad or truncate to a fixed length (e.g., 173, or any desired length)
    target_length = 173  # You can adjust this value
    if mfccs.shape[1] < target_length:
        mfccs = np.pad(mfccs, ((0, 0), (0, target_length - mfccs.shape[1])), mode='constant')
    elif mfccs.shape[1] > target_length:
        mfccs = mfccs[:, :target_length]

    return mfccs

# Feature extraction for Pneumonia audio
pneumonia_features = []
for filename in os.listdir(pneumonia_audio_dir):
    if filename.endswith('.wav'):
        file_path = os.path.join(pneumonia_audio_dir, filename)
        mfccs = extract_features(file_path)
        pneumonia_features.append(mfccs)

# Feature extraction for Normal audio
normal_features = []
for filename in os.listdir(normal_audio_dir):
    if filename.endswith('.wav'):
        file_path = os.path.join(normal_audio_dir, filename)
        mfccs = extract_features(file_path)
        normal_features.append(mfccs)

# Save the extracted features as NumPy arrays
np.save('/content/drive/MyDrive/Features/Pneumonia_Features.npy', np.array(pneumonia_features))
np.save('/content/drive/MyDrive/Features/Normal_Features.npy', np.array(normal_features))


In [ ]:
import os
import random
from sklearn.model_selection import train_test_split

# Define the paths to your audio data directories
pneumonia_dir = '/content/drive/MyDrive/New Audio Folder/Pneumonia'
normal_dir = '/content/drive/MyDrive/New Audio Folder/Normal'

# List the audio files in each directory
pneumonia_files = [os.path.join(pneumonia_dir, filename) for filename in os.listdir(pneumonia_dir) if filename.endswith('.wav')]
normal_files = [os.path.join(normal_dir, filename) for filename in os.listdir(normal_dir) if filename.endswith('.wav')]

# Shuffle the data
random.shuffle(pneumonia_files)
random.shuffle(normal_files)

# Define split ratios (e.g., 80% training, 10% validation, 10% test)
train_ratio = 0.8
val_ratio = 0.1

# Split the data into training, validation, and test sets
pneumonia_train, pneumonia_test = train_test_split(pneumonia_files, test_size=(1 - train_ratio))
pneumonia_val, pneumonia_test = train_test_split(pneumonia_test, test_size=(val_ratio / (1 - train_ratio)))

normal_train, normal_test = train_test_split(normal_files, test_size=(1 - train_ratio))
normal_val, normal_test = train_test_split(normal_test, test_size=(val_ratio / (1 - train_ratio)))

# Combine data from both classes
train_data = pneumonia_train + normal_train
val_data = pneumonia_val + normal_val
test_data = pneumonia_test + normal_test


**CNN Model**

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Define your CNN model
target_length = 173  # Replace with the actual maximum MFCC feature length
NUM_CLASSES = 2  # Define the number of classes in your dataset (Normal and Pneumonia)

input_shape = (MFCC_FEATURES, target_length, 1)
model = keras.Sequential([
    # Convolutional layers
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    keras.layers.MaxPooling2D((2, 2)),

    # Flatten and Dense layers
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(NUM_CLASSES, activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary to see the architecture
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 13, 173, 32)       320       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 6, 86, 32)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 6, 86, 64)         18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 3, 43, 64)         0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 8256)              0         
                                                                 
 dense_3 (Dense)             (None, 128)              

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
16/16 [==============================] - 3s 112ms/step - loss: 12.4458 - accuracy: 0.5351 - val_loss: 0.9166 - val_accuracy: 0.5865
Epoch 2/10
16/16 [==============================] - 2s 103ms/step - loss: 0.6345 - accuracy: 0.6570 - val_loss: 0.6026 - val_accuracy: 0.6923
Epoch 3/10
16/16 [==============================] - 2s 99ms/step - loss: 0.4923 - accuracy: 0.7665 - val_loss: 0.5521 - val_accuracy: 0.6827
Epoch 4/10
16/16 [==============================] - 2s 99ms/step - loss: 0.3312 - accuracy: 0.8822 - val_loss: 0.4543 - val_accuracy: 0.7981
Epoch 5/10
16/16 [==============================] - 2s 105ms/step - loss: 0.1896 - accuracy: 0.9421 - val_loss: 0.3145 - val_accuracy: 0.8558
Epoch 6/10
16/16 [==============================] - 2s 147ms/step - loss: 0.1180 - accuracy: 0.9731 - val_loss: 0.2287 - val_accuracy: 0.8942
Epoch 7/10
16/16 [==============================] - 3s 209ms/step - loss: 0.0676 - accuracy: 0.9855 - val_loss: 0.1782 - val_accuracy: 0.9327
Epoch 8

In [ ]:
X_test = X_test.astype(np.float32)  # Ensure that X_test is in float32
y_test = y_test.astype(np.int32)    # Ensure that y_test is in int32

# Now you can evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')


4/4 [==============================] - 1s 52ms/step - loss: 0.1251 - accuracy: 0.9519
Test accuracy: 0.9519230723381042


In [ ]:
# Specify the path where you want to save the model
model_save_path = '/content/drive/MyDrive/My_model.h5'  # Replace 'Your_Model_Name' with the desired name

# Save the model
model.save(model_save_path)

print(f"Model saved to {model_save_path}")


Model saved to /content/drive/MyDrive/My_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
import librosa
import numpy as np

# Constants for feature extraction
SAMPLE_RATE = 22050
DURATION = 4
MFCC_FEATURES = 13

# Function to extract MFCC features from an audio clip
def extract_features(file_path, target_length):
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    mfccs = librosa.feature.mfcc(y=audio, sr=SAMPLE_RATE, n_mfcc=MFCC_FEATURES)

    # Pad or truncate to the target length
    if mfccs.shape[1] < target_length:
        mfccs = np.pad(mfccs, ((0, 0), (0, target_length - mfccs.shape[1])), mode='constant')
    elif mfccs.shape[1] > target_length:
        mfccs = mfccs[:, :target_length]

    return mfccs

# Path to your audio clip
audio_clip_path = '/content/drive/MyDrive/New Audio Folder/Pneumonia/Copy of P10_noise.wav'

# Load the saved model
model_load_path = '/content/drive/MyDrive/My_model.h5'  # Specify the path to your saved model
loaded_model = tf.keras.models.load_model(model_load_path)

# Extract MFCC features from the audio clip
target_length = 173  # Replace with your target length
audio_features = extract_features(audio_clip_path, target_length)

# Reshape the audio features to match the model's input shape
audio_features = audio_features.reshape(1, MFCC_FEATURES, target_length, 1)

# Make predictions
predictions = loaded_model.predict(audio_features)

# The predictions will be a probability distribution over your classes. You can interpret the results.
# In this example, we'll assume a binary classification (Normal and Pneumonia).
if predictions[0][0] > predictions[0][1]:
    predicted_class = 'Normal'
else:
    predicted_class = 'Pneumonia'

print(f'Predicted Class: {predicted_class}')


1/1 [==============================] - 0s 167ms/step
Predicted Class: Pneumonia


In [2]:
import tensorflow as tf
import librosa
import numpy as np

# Constants for feature extraction
SAMPLE_RATE = 22050
DURATION = 4
MFCC_FEATURES = 13

# Function to extract MFCC features from an audio clip
def extract_features(file_path, target_length):
    audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, duration=DURATION)
    mfccs = librosa.feature.mfcc(y=audio, sr=SAMPLE_RATE, n_mfcc=MFCC_FEATURES)

    # Pad or truncate to the target length
    if mfccs.shape[1] < target_length:
        mfccs = np.pad(mfccs, ((0, 0), (0, target_length - mfccs.shape[1])), mode='constant')
    elif mfccs.shape[1] > target_length:
        mfccs = mfccs[:, :target_length]

    return mfccs

# Path to your audio clip
audio_clip_path = '/content/drive/MyDrive/New Audio Folder/Normal/Copy of B11_pitch_shift.wav'

# Load the saved model
model_load_path = '/content/drive/MyDrive/My_model.h5'  # Specify the path to your saved model
loaded_model = tf.keras.models.load_model(model_load_path)

# Extract MFCC features from the audio clip
target_length = 173  # Replace with your target length
audio_features = extract_features(audio_clip_path, target_length)

# Reshape the audio features to match the model's input shape
audio_features = audio_features.reshape(1, MFCC_FEATURES, target_length, 1)

# Make predictions
predictions = loaded_model.predict(audio_features)

# The predictions will be a probability distribution over your classes. You can interpret the results.
# In this example, we'll assume a binary classification (Normal and Pneumonia).
if predictions[0][0] > predictions[0][1]:
    predicted_class = 'Normal'
else:
    predicted_class = 'Pneumonia'

print(f'Predicted Class: {predicted_class}')


1/1 [==============================] - 0s 101ms/step
Predicted Class: Normal
